# Using the Linux GPIO character device and LIBGPIOD

---
## Disclaimer:
Because:
- I am only using one dynamically linked file, AND
- I haven't created a separate disk for cross copiled linked libraries, AND
- quite frankly I have NOT yet found a sensible comprehensible useful human readable step by step description how to set up and use "pkg-config" with cross compilation..

I came up with a simplified way of doing this

---
## Installing LIBGPIOD on the Host
**NOTE:** All of this is run on the host development PC

I made the libgpiod project part of my project on my **HOST** development machine. The git repo is **already** linked as a git submodule in the BBB3 project. If you go into the root of the BBB3 project (/workspace/BBB3) you will see an empty directory called libgpiod. I did this because want my project to stay up to date with libgpiod project without separate git management or cross contamination.

The directory is empty, this is because the git submodule needs to be initialised **once**:
```
cd /workspace/BBB3
git submodule init
git submodule update
```
The this point the directory will be populated.

---
## Install LIBGPIOD on the BBB3 target

**NOTE:** All of this is run on the BBB3 target device..

```
cd ~
mkdir builds
cd builds
sudo apt-get install -y libtool pkg-config autoconf-archive
sudo apt-get install -y build-essential git
git clone git://git.kernel.org/pub/scm/libs/libgpiod/libgpiod.git
```

### Compile the library and tools
There is a step in here to create an "m4" durectory. This prevents occasional autoconf errors.

**Note:**
- this builds in the CPP (cxx) bindings
- this does not build the python bindings. To add theese you would add "--enable-bindings-python" at the configure stage
```
cd libgpiod
mkdir m4
./autogen.sh --enable-tools=yes --enable-bindings-cxx --prefix=/usr/local
make
sudo make install
sudo ldconfig
```

### Test the installation
Test for the header files:
```
pkg-config --cflags libgpiod
```
This should return:
```
-I/usr/local/include
```

Test for the library:
```
pkg-config --lib libgpiod
```
This should return:
```
-L/usr/local/lib -lgpiod
```

---
## Set up the host environment

This is a **simple hack**. On the **Linux host** (i.e. the cross compile environment) copy all the libgpiod files from the BBB3 target to the host. This lets the cross compiling work without needing to mess with pkg-config paths.

```
cd /usr/local/lib
sudo scp debian@192.168.1.65:/usr/local/lib/libgpiod.* .
```
  
The rest is managed in the makefile and the BBB3 repository by refering to the linked libgpiod subproject